#**Data Cleaning**
Master efficient workflows for cleaning real-world, messy data.

https://www.kaggle.com/learn/data-cleaning

---
**Autor:** Marcos Bezerra

**GitHub:** [https://github.com/marcos-bezerra/Data_Cleaning_Kaggle](https://github.com/marcos-bezerra/Data_Cleaning_Kaggle)

**Google Drive:** [https://drive.google.com/marcos-bezerra/Data_Cleaning_Kaggle](https://drive.google.com/drive/folders/1eHyIT60C7-QV_DaCjMFHBLAaSxbl1ikN?usp=sharing)

**Versão:** 1.0 - 13 Fev 2022

---

## **Lesson 05 - Inconsistent Data Entry**

Efficiently fix typos in your data.

In [ ]:
# Autorização para acessar o google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# informa o path da Pasta Raiz da pasta de trabalho
import os
pathRaiz = '/content/drive/MyDrive/I2A2/Desafio_03_Kaggle_DataClean'
os.chdir(pathRaiz)
os.getcwd()

'/content/drive/MyDrive/I2A2/Desafio_03_Kaggle_DataClean'

In [ ]:
ls -lah dataset

total 429M
-rw------- 1 root root 432K Feb 12 12:42  catalog.csv
-rw------- 1 root root  45M Feb 13 12:33  ks-projects-201612.csv
-rw------- 1 root root  47M Feb 13 12:37  ks-projects-201612-utf8.csv
-rw------- 1 root root 263M Sep 20  2019 'NFL Play by Play 2009-2017 (v4).csv'
-rw------- 1 root root  76M Feb  9 11:51 'NFL Play by Play 2009-2017 (v4).csv.zip'
-rw------- 1 root root 231K Feb 12 12:40  pakistan_intellectual_capital.csv


# Configure nosso ambiente
A primeira coisa que precisamos fazer é carregar as bibliotecas e o conjunto de dados que usaremos.

In [ ]:
!pip install Fuzzywuzzy

# modules we'll use
import pandas as pd
import numpy as np

# helpful modules
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

# read in all our data
professors = pd.read_csv("dataset/pakistan_intellectual_capital.csv")

# set seed for reproducibility
np.random.seed(0)

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Faça algum pré-processamento de texto preliminar
Começaremos dando uma rápida olhada nas primeiras linhas dos dados.

In [ ]:
professors.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,Pakistan,2005.0,VLSI Electronics DLD Database,NaN


Digamos que estamos interessados em limpar a coluna "País" para garantir que não haja inconsistências na entrada de dados. Poderíamos examinar e verificar cada linha à mão, é claro, e corrigir inconsistências à mão quando as encontrarmos. Há uma maneira mais eficiente de fazer isso, no entanto!

In [ ]:
# get all the unique values in the 'Country' column
countries = professors['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array([' Germany', ' New Zealand', ' Sweden', ' USA', 'Australia',
       'Austria', 'Canada', 'China', 'Finland', 'France', 'Greece',
       'HongKong', 'Ireland', 'Italy', 'Japan', 'Macau', 'Malaysia',
       'Mauritius', 'Netherland', 'New Zealand', 'Norway', 'Pakistan',
       'Portugal', 'Russian Federation', 'Saudi Arabia', 'Scotland',
       'Singapore', 'South Korea', 'SouthKorea', 'Spain', 'Sweden',
       'Thailand', 'Turkey', 'UK', 'USA', 'USofA', 'Urbana', 'germany'],
      dtype=object)

Apenas olhando para isso, posso ver alguns problemas devido à entrada de dados inconsistente: 'Alemanha' e 'alemanha', por exemplo, ou 'Nova Zelândia' e 'Nova Zelândia'.

A primeira coisa que vou fazer é deixar tudo em letras minúsculas (posso mudar de volta no final se quiser) e remover quaisquer espaços em branco no início e no final das células. Inconsistências em maiúsculas e espaços em branco à direita são muito comuns em dados de texto e você pode corrigir cerca de 80% das inconsistências de entrada de dados de texto fazendo isso.

In [ ]:
# convert to lower case
professors['Country'] = professors['Country'].str.lower()
# remove trailing white spaces
professors['Country'] = professors['Country'].str.strip()

Em seguida, vamos lidar com inconsistências mais difíceis.

**Use fuzzy matching to correct inconsistent data entry**

Tudo bem, vamos dar outra olhada na coluna 'País' e ver se há mais alguma limpeza de dados que precisamos fazer.

In [ ]:
# get all the unique values in the 'Country' column
countries = professors['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea',
       'southkorea', 'spain', 'sweden', 'thailand', 'turkey', 'uk',
       'urbana', 'usa', 'usofa'], dtype=object)

Parece que há outra inconsistência: 'Coreia do Sul' e 'Coreia do Sul' devem ser iguais.

Vamos usar o pacote **fuzzywuzzy** para ajudar a identificar quais strings estão mais próximas umas das outras. Esse conjunto de dados é pequeno o suficiente para que provavelmente possamos corrigir erros manualmente, mas essa abordagem não é bem dimensionada. (Você gostaria de corrigir mil erros à mão? E dez mil? Automatizar as coisas o mais cedo possível geralmente é uma boa ideia. Além disso, é divertido!)

**Fuzzy matching**: O processo de encontrar automaticamente strings de texto muito semelhantes à string de destino. Em geral, uma string é considerada "mais próxima" de outra quanto menos caracteres você precisaria alterar se estivesse transformando uma string em outra. Portanto, "apple" e "snapple" são duas alterações de distância uma da outra (adicione "s" e "n") enquanto "in" e "on" e uma alteração de distância (rplace "i" por "o"). Você nem sempre poderá confiar na correspondência difusa 100%, mas geralmente acabará economizando pelo menos um pouco de tempo.

**Fuzzywuzzy** retorna uma proporção dada duas strings. Quanto mais próxima a proporção estiver de 100, menor será a distância de edição entre as duas cordas. Aqui, vamos pegar as dez strings da nossa lista de cidades que têm a distância mais próxima de "d.i khan".

In [ ]:
# get the top 10 closest matches to "south korea"
matches = fuzzywuzzy.process.extract("south korea", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# take a look at them
matches

[('south korea', 100),
 ('southkorea', 48),
 ('saudi arabia', 43),
 ('norway', 35),
 ('ireland', 33),
 ('portugal', 32),
 ('singapore', 30),
 ('netherland', 29),
 ('macau', 25),
 ('usofa', 25)]

Podemos ver que dois dos itens nas cidades estão muito próximos da "Coreia do Sul": "Coreia do Sul" e "Coreia do Sul". Vamos substituir todas as linhas em nossa coluna "País" que tenham uma proporção > 47 por "Coreia do Sul".

Para fazer isso, vou escrever uma função. (É uma boa ideia escrever uma função de propósito geral que você possa reutilizar se achar que pode ter que fazer uma tarefa específica mais de uma ou duas vezes. Isso evita que você tenha que copiar e colar código com muita frequência, o que economiza tempo e pode ajudar evitar erros.)

In [ ]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

Now that we have a function, we can put it to the test!

In [ ]:
# use the function we just wrote to replace close matches to "south korea" with "south korea"
replace_matches_in_column(df=professors, column='Country', string_to_match="south korea")

All done!


E agora vamos verificar os valores exclusivos em nossa coluna "País" novamente e ter certeza de que arrumamos "Coreia do Sul" corretamente.

In [ ]:
# get all the unique values in the 'Country' column
countries = professors['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'urbana', 'usa', 'usofa'],
      dtype=object)

Excelente! Agora temos apenas "Coreia do Sul" em nosso dataframe e não precisamos alterar nada manualmente.